we want to combine full-text score with popularity

In [1]:
curl -XPUT 'localhost:9200/blogposts/post/1?pretty' -H 'Content-Type: application/json' -d'
{
  "title":   "About popularity",
  "content": "In this post we will talk about...",
  "votes":   6
}
'

{
  "_index" : "blogposts",
  "_type" : "post",
  "_id" : "1",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "failed" : 0
  },
  "created" : true
}


use the function_score query to do this

In [4]:
curl -XGET 'localhost:9200/blogposts/post/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "function_score": { 
      "query": { 
        "multi_match": {
          "query":    "popularity",
          "fields": [ "title", "content" ]
        }
      },
      "field_value_factor": { 
        "field": "votes" 
      }
    }
  }
}
'

{
  "took" : 8,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 1.5486964,
    "hits" : [
      {
        "_index" : "blogposts",
        "_type" : "post",
        "_id" : "1",
        "_score" : 1.5486964,
        "_source" : {
          "title" : "About popularity",
          "content" : "In this post we will talk about...",
          "votes" : 6
        }
      }
    ]
  }
}


votes are counting for way too much, need to smooth it out

In [5]:
curl -XGET 'localhost:9200/blogposts/post/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "function_score": {
      "query": {
        "multi_match": {
          "query":    "popularity",
          "fields": [ "title", "content" ]
        }
      },
      "field_value_factor": {
        "field":    "votes",
        "modifier": "log1p" 
      }
    }
  }
}
'

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.21813338,
    "hits" : [
      {
        "_index" : "blogposts",
        "_type" : "post",
        "_id" : "1",
        "_score" : 0.21813338,
        "_source" : {
          "title" : "About popularity",
          "content" : "In this post we will talk about...",
          "votes" : 6
        }
      }
    ]
  }
}


we can increase the votes strength by adding a factor

In [6]:
curl -XGET 'localhost:9200/blogposts/post/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "function_score": {
      "query": {
        "multi_match": {
          "query":    "popularity",
          "fields": [ "title", "content" ]
        }
      },
      "field_value_factor": {
        "field":    "votes",
        "modifier": "log1p",
        "factor":   2 
      }
    }
  }
}
'

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.28752667,
    "hits" : [
      {
        "_index" : "blogposts",
        "_type" : "post",
        "_id" : "1",
        "_score" : 0.28752667,
        "_source" : {
          "title" : "About popularity",
          "content" : "In this post we will talk about...",
          "votes" : 6
        }
      }
    ]
  }
}


simmer down weight for votes, we can add a boost mode to add the scores and not multiply

In [7]:
curl -XGET 'localhost:9200/blogposts/post/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "function_score": {
      "query": {
        "multi_match": {
          "query":    "popularity",
          "fields": [ "title", "content" ]
        }
      },
      "field_value_factor": {
        "field":    "votes",
        "modifier": "log1p",
        "factor":   0.1
      },
      "boost_mode": "sum" 
    }
  }
}
'

{
  "took" : 6,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.46223605,
    "hits" : [
      {
        "_index" : "blogposts",
        "_type" : "post",
        "_id" : "1",
        "_score" : 0.46223605,
        "_source" : {
          "title" : "About popularity",
          "content" : "In this post we will talk about...",
          "votes" : 6
        }
      }
    ]
  }
}


we can cap the field_value_factor value but applying a max_boost

In [8]:
curl -XGET 'localhost:9200/blogposts/post/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "function_score": {
      "query": {
        "multi_match": {
          "query":    "popularity",
          "fields": [ "title", "content" ]
        }
      },
      "field_value_factor": {
        "field":    "votes",
        "modifier": "log1p",
        "factor":   0.1
      },
      "boost_mode": "sum",
      "max_boost":  1.5 
    }
  }
}
'

{
  "took" : 4,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.46223605,
    "hits" : [
      {
        "_index" : "blogposts",
        "_type" : "post",
        "_id" : "1",
        "_score" : 0.46223605,
        "_source" : {
          "title" : "About popularity",
          "content" : "In this post we will talk about...",
          "votes" : 6
        }
      }
    ]
  }
}
